In [1]:
import pandas as pd
import numpy as np
     
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from nltk.tokenize import word_tokenize 
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

!pip install emoji
import emoji
!pip install openai
import openai
nltk.download('punkt')
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore")
!pip install -q keras
import keras
from os import path
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print(accelerator)
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.models import Sequential
from keras import layers
from google.colab import drive
drive.mount('/content/drive')
import csv



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


cu80
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Reading

In [ ]:
# 导入训练集数据 土耳其语
olid_training=pd.read_csv("/content/drive/MyDrive/TURKISH-DATA/offenseval-tr-training-v1.tsv",sep="\t")
X_train_FULL=olid_training[["id","tweet","subtask_a"]] 
Y_train_FULL=olid_training["subtask_a"]
# 导入测试数据及结果
X_test_FULL=pd.read_csv('/content/drive/MyDrive/TURKISH-DATA/offenseval-tr-testset-v1.tsv',sep='\t',encoding='utf8',quoting=csv.QUOTE_NONE)
Y_TEST=pd.read_csv('/content/drive/MyDrive/TURKISH-DATA/offenseval-tr-labela-v1.tsv',sep=',',encoding='utf8',quoting=csv.QUOTE_NONE,header=None)
Y_TRAIN_ENCODED_FULL=[1 if i ==  'OFF' else 0 for i in Y_train_FULL]
Y_TEST_ENCODED_FULL = [1 if i ==  'OFF' else 0 for i in Y_TEST[1]]

print("---导入数据成功---")



---导入数据成功---


In [2]:
# 导入训练集数据 英语
olid_training=pd.read_csv("/content/drive/MyDrive/OLIDv1.0/olid-training-v1.0.tsv",sep="\t")
X_train_FULL=olid_training[["id","tweet","subtask_a"]] 
Y_train_FULL=olid_training["subtask_a"]
# 导入测试数据及结果
X_test_FULL=pd.read_csv('/content/drive/MyDrive/OLIDv1.0/testset-levela.tsv',sep='\t',encoding='utf8',quoting=csv.QUOTE_NONE)
Y_TEST=pd.read_csv('/content/drive/MyDrive/OLIDv1.0/labels-levela.csv',sep=',',encoding='utf8',quoting=csv.QUOTE_NONE,header=None)
Y_TRAIN_ENCODED_FULL=[1 if i ==  'OFF' else 0 for i in Y_train_FULL]
Y_TEST_ENCODED_FULL = [1 if i ==  'OFF' else 0 for i in Y_TEST[1]]

print("---导入数据成功---")


---导入数据成功---


Pre-processing

In [3]:
def preprocess_tweet(tweet):
    # Split tweet into tokens
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(tweet)

    # Remove URLs and mentions
    tokens = [token for token in tokens if not token.startswith('http') and not token.startswith('@')]

    # Decode emojis
    tokens = [emoji.demojize(token) for token in tokens]

    # Remove punctuation and non-alphanumeric characters
    tokens = [token for token in tokens if token.isalnum()]

    # Convert all tokens to lowercase
    tokens = [token.lower() for token in tokens]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    tokens = [token for token in tokens if ((token != "url"))]
    return tokens

In [4]:
# 数据清洗

filtered_tweets=[]
for tweet in X_train_FULL["tweet"]:
    filtered_tweets.append(preprocess_tweet(tweet))
X_train_FULL["tweet_initial"] = filtered_tweets


filtered_tweets=[]
for tweet in X_test_FULL["tweet"]:
    filtered_tweets.append(preprocess_tweet(tweet))
X_test_FULL["tweet_initial"] = filtered_tweets




#FOR TEST

z=[]
for tweet in X_train_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_train_FULL["tweet_initial_nontoken"]=z



#FOR TEST

z=[]
for tweet in X_test_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_test_FULL["tweet_initial_nontoken"]=z



In [5]:
X_test_FULL

,id,tweet,tweet_initial,tweet_initial_nontoken
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,"[democrats, support, antifa, muslim, brotherho...",democrats support antifa muslim brotherhood ms...
1,27014,"#ConstitutionDay is revered by Conservatives, ...","[revered, conservatives, hated, progressives, ...",revered conservatives hated progressives socia...
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,"[first, reduces, ca]",first reduces ca
3,13876,#Watching #Boomer getting the news that she is...,"[getting, news, still, parole, always, makes, ...",getting news still parole always makes smile f...
4,60133,#NoPasaran: Unity demo to oppose the far-right...,"[unity, demo, oppose, enough, enough]",unity demo oppose enough enough
...,...,...,...,...
855,73439,#DespicableDems lie again about rifles. Dem Di...,"[lie, rifles, dem, distorted, law, push, kavan...",lie rifles dem distorted law push kavanaugh co...
856,25657,#MeetTheSpeakers 🙌 @USER will present in our e...,"[present, event, oiw, 2018, finpact, global, i...",present event oiw 2018 finpact global impact f...
857,67018,3 people just unfollowed me for talking about ...,"[3, people, unfollowed, talking, merlin, sorry...",3 people unfollowed talking merlin sorry im st...
858,50665,#WednesdayWisdom Antifa calls the right fascis...,"[antifa, calls, right, fascist, reality, left,...",antifa calls right fascist reality left follow...


In [5]:
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers import Embedding
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
def recall_m(true_Y, pred_Y):
        TP = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        possible_pos = K.sum(K.round(K.clip(true_Y, 0, 1)))
        rec = TP / (possible_pos + K.epsilon())
        return rec

def precision_m(true_Y, pred_Y):
        true_positives = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(pred_Y, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision


    

def f1_m(true_Y, pred_Y):
    pres = precision_m(true_Y, pred_Y)
    rec = recall_m(true_Y, pred_Y)
    return 2*((pres*rec)/(pres+rec+K.epsilon()))

from keras.models import Sequential
from keras import layers


## Tweeter Word2vec / Custom Embedding Layer

In [ ]:
fname= "/content/drive/My Drive/Twitter/Word2Vec/w2v_model_word.vec"


import gensim
from gensim.models import Word2Vec
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(fname)  # you can load this saved keyedvectors model later



# Tweeter FastText

In [8]:
import gensim
from gensim.models import FastText

word_vectors = gensim.models.FastText.load_fasttext_format('/content/drive/MyDrive/cc.en.300.bin',encoding='utf-8') # use that if you want to use fasttedxt 



# Public FastText

In [ ]:
word_vectors = gensim.models.FastText.load_fasttext_format('/content/drive/My Drive/OFFENSEVAL20-DATA/haber-P1_S0_L0.bin',encoding='utf-8') # use that if you want to use fasttedxt 


# Public word2vec

In [6]:

fname= "/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz"

import gensim
from gensim.models import Word2Vec

word_vectors = gensim.models.KeyedVectors.load_word2vec_format(fname, binary = True)


## Tokenizing / creating vocabulary and wordindex using keras functinalities

In [7]:
"""
We will use word indexes as look-up table during embedding layer.
"""
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()  #the maximum number of words to keep, based on word frequency. Only the most common num_words-1 words will be kept.
# tokenizer = Tokenizer(num_words=98790)  #the maximum number of words to keep, based on word frequency. Only the most common num_words-1 words will be kept.
tokenizer.fit_on_texts(X_train_FULL["tweet_initial_nontoken"])
X_train_initial = tokenizer.texts_to_sequences(X_train_FULL["tweet_initial_nontoken"])
X_test_initial = tokenizer.texts_to_sequences(X_test_FULL["tweet_initial_nontoken"])
vocab_size_initial = len(tokenizer.word_index) + 1 
wordIndex_initial=tokenizer.word_index # it is  index
from keras_preprocessing.sequence import pad_sequences
max_len = 30

"""
Padding

"""
X_train_initial = pad_sequences(X_train_initial, padding='post', maxlen=max_len)
X_test_initial = pad_sequences(X_test_initial, padding='post', maxlen=max_len)




In [8]:
def createEmbeddingLayer(wordIndex,not_static):
  a=[]
  embedding_dim=300
  vocabulary_size=len(wordIndex)+1
  embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
  missed=0
  for word, i in wordIndex.items():
    
          
      try:
          embedding_vector = word_vectors[word] # or fast text
          embedding_matrix[i] = embedding_vector
            

      except KeyError: # If word is not found in the word2vec vocabulary , assign random weights
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),embedding_dim)
        missed+=1
        a.append(word)

  print('missed_words :' , missed)

  custom_embedding_layer = Embedding(vocabulary_size,
                                embedding_dim,
                                weights=[embedding_matrix],
                                trainable=not_static )# Controls the updating weights )
  return custom_embedding_layer

  

In [13]:
from sklearn.metrics import f1_score
import pickle

def cnn(vocab_size,X_train,X_test,y_train,y_test,wordIndex,trainable):
  early_stopping = [EarlyStopping(monitor='val_loss',
                        min_delta=0,restore_best_weights=True,
                        patience=5,
                        verbose=1, mode='auto')]
  model = Sequential()
  model.add(createEmbeddingLayer(wordIndex,trainable))
  model.add(Dropout(0.2))
  #model.add(Conv1D(128, 1, activation='relu'))
  model.add(Conv1D(128, 3, activation='relu'))
  model.add(layers.GlobalMaxPooling1D())
  
  #model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
  model.add(layers.Dense(100, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False), metrics=['acc',f1_m,precision_m, recall_m])
  ## Fit the model
  model.summary()
  model.fit(X_train, y_train, validation_split=0.1, epochs=20,callbacks=early_stopping,batch_size=32)
  loss, accuracy, f1_score, precision, recall = model.evaluate(X_train, y_train, verbose=1)
  print("cnn Training Loss: {:.4f}".format(loss))
  print("cnn Training Accuracy: {:.4f}".format(accuracy))
  print("cnn Training f1 score: {:.4f}".format(f1_score))
  print("cnn Training Precision: {:.4f}".format(precision))
  print("cnn Training Recall: {:.4f}".format(recall))

  loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)
  print("cnn Test Loss: {:.4f}".format(loss))
  print("cnn Test Accuracy: {:.4f}".format(accuracy))
  print("cnn Test f1 score: {:.4f}".format(f1_score))
  print("cnn Test Precision: {:.4f}".format(precision))
  print("cnn Test Recall: {:.4f}".format(recall))

  probs = model.predict(X_test, verbose=1)
  predicted_classes = (model.predict(X_test) > 0.5).astype("int32")


  #filename = 'finalized_model_lstm.sav'
  #pickle.dump(model, open(filename, 'wb'))

  print(classification_report(y_test, predicted_classes ,digits=3 ))


  print("cnn  ends..")
  return (predicted_classes,probs)

In [10]:
X_train_FULL = np.array(X_train_FULL)
X_test_FULL = np.array(X_test_FULL)
Y_TRAIN_ENCODED_FULL = np.array(Y_TRAIN_ENCODED_FULL)
Y_TEST_ENCODED_FULL = np.array(Y_TEST_ENCODED_FULL)

In [17]:
prediction_cnnlstm,probs_cnnlstm=cnn(vocab_size_initial,X_train_initial,X_test_initial,Y_TRAIN_ENCODED_FULL,Y_TEST_ENCODED_FULL,wordIndex_initial,False)

missed_words : 2607
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 300)         5155800   
                                                                 
 dropout_4 (Dropout)         (None, None, 300)         0         
                                                                 
 conv1d_4 (Conv1D)           (None, None, 128)         115328    
                                                                 
 global_max_pooling1d_4 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_8 (Dense)             (None, 100)               12900     
                                                                 
 dense_9 (Dense)             (None, 1)                 101       
                                  

In [19]:
from sklearn.metrics import f1_score
import pickle
from keras.layers import GRU

def gru(vocab_size,X_train,X_test,y_train,y_test,wordIndex,trainable):
  early_stopping = [EarlyStopping(monitor='val_loss',
                        min_delta=0,restore_best_weights=True,
                        patience=5,
                        verbose=1, mode='auto')]
  model = Sequential()
  model.add(createEmbeddingLayer(wordIndex,trainable))
  model.add(Dropout(0.2))
  model.add(GRU(32))
  #model.add(Conv1D(128, 1, activation='relu'))
  # model.add(Conv1D(128, 3, activation='relu'))
  # model.add(layers.GlobalMaxPooling1D())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))
  #model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
  # model.add(layers.Dense(100, activation='sigmoid'))
  model.add(Dense(1, activation='sigmoid'))
  model.summary()
  model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False), metrics=['acc',f1_m,precision_m, recall_m])
  ## Fit the model
  model.fit(X_train, y_train, validation_split=0.1, epochs=20,callbacks=early_stopping,batch_size=32)
  loss, accuracy, f1_score, precision, recall = model.evaluate(X_train, y_train, verbose=1)
  print("cnn Training Loss: {:.4f}".format(loss))
  print("cnn Training Accuracy: {:.4f}".format(accuracy))
  print("cnn Training f1 score: {:.4f}".format(f1_score))
  print("cnn Training Precision: {:.4f}".format(precision))
  print("cnn Training Recall: {:.4f}".format(recall))

  loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)
  print("cnn Test Loss: {:.4f}".format(loss))
  print("cnn Test Accuracy: {:.4f}".format(accuracy))
  print("cnn Test f1 score: {:.4f}".format(f1_score))
  print("cnn Test Precision: {:.4f}".format(precision))
  print("cnn Test Recall: {:.4f}".format(recall))

  probs = model.predict(X_test, verbose=1)
  predicted_classes = (model.predict(X_test) > 0.5).astype("int32")


  #filename = 'finalized_model_lstm.sav'
  #pickle.dump(model, open(filename, 'wb'))

  print(classification_report(y_test, predicted_classes ,digits=3 ))


  print("cnn  ends..")
  return (predicted_classes,probs)

In [20]:
prediction_cnnlstm,probs_cnnlstm=gru(vocab_size_initial,X_train_initial,X_test_initial,Y_TRAIN_ENCODED_FULL,Y_TEST_ENCODED_FULL,wordIndex_initial,False)

missed_words : 2607
Epoch 1/20
373/373 [==============================] - 6s 8ms/step - loss: 0.5743 - acc: 0.7148 - f1_m: 0.3393 - precision_m: 0.4035 - recall_m: 0.3288 - val_loss: 0.4731 - val_acc: 0.7847 - val_f1_m: 0.6597 - val_precision_m: 0.6770 - val_recall_m: 0.6602
Epoch 2/20
373/373 [==============================] - 2s 6ms/step - loss: 0.4790 - acc: 0.7769 - f1_m: 0.6236 - precision_m: 0.6970 - recall_m: 0.5951 - val_loss: 0.4530 - val_acc: 0.7983 - val_f1_m: 0.6563 - val_precision_m: 0.7452 - val_recall_m: 0.6026
Epoch 3/20
373/373 [==============================] - 2s 6ms/step - loss: 0.4531 - acc: 0.7882 - f1_m: 0.6379 - precision_m: 0.7253 - recall_m: 0.5974 - val_loss: 0.4773 - val_acc: 0.7560 - val_f1_m: 0.6637 - val_precision_m: 0.5991 - val_recall_m: 0.7608
Epoch 4/20
373/373 [==============================] - 2s 6ms/step - loss: 0.4464 - acc: 0.7950 - f1_m: 0.6517 - precision_m: 0.7324 - recall_m: 0.6123 - val_loss: 0.4886 - val_acc: 0.7477 - val_f1_m: 0.6621 - val

In [21]:
from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding, Dense, Dropout, Bidirectional
from tensorflow.keras.layers import Bidirectional,LSTM

def sigmoid(x): 
    return 1.0/(1 + np.exp(-x))


class biLSTM(object):
    def __init__(self, maxlen, embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        self.maxlen = maxlen
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))
        embedding = createEmbeddingLayer(wordIndex_initial,False)(input) #change
        x = Bidirectional(LSTM(128, return_sequences=False))(embedding)  # LSTM or GRU
        x = Dense(128, activation='relu')(x)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model

def bilstm(X_train_initial,X_test_initial,y_train,y_test) : 
  LSTM_MODEL= biLSTM(30, 100).get_model()

  early_stopping = [EarlyStopping(monitor='val_loss',min_delta=0,restore_best_weights=True, patience=10,verbose=1, mode='auto')]
  LSTM_MODEL.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['acc',f1_m,precision_m, recall_m])
  LSTM_MODEL.summary()

  LSTM_MODEL.fit(X_train_initial, y_train,
            batch_size=16,
            epochs=30,
            callbacks=early_stopping,
            validation_split=0.1)

  loss, accuracy, f1_score, precision, recall = LSTM_MODEL.evaluate(X_train_initial, y_train, verbose=1)
  print(" biLSTM Training Loss: {:.4f}".format(loss))
  print(" biLSTM Accuracy: {:.4f}".format(accuracy))
  print("  biLSTM  f1 score: {:.4f}".format(f1_score))
  print("Attention- BiRNN Precision: {:.4f}".format(precision))
  print("Attention- BiRNN Recall: {:.4f}".format(recall))

  loss, accuracy, f1_score, precision, recall = LSTM_MODEL.evaluate(X_test_initial, y_test, verbose=1)
  print(" biLSTM Test Loss: {:.4f}".format(loss))
  print(" biLSTM Test Accuracy: {:.4f}".format(accuracy))
  print("  biLSTM Test f1 score: {:.4f}".format(f1_score))
  print(" biLSTM  Test Precision: {:.4f}".format(precision))
  print(" biLSTM Test Recall: {:.4f}".format(recall))
 
  probs = LSTM_MODEL.predict(X_test_initial, verbose=1)
  print('lenght of probs : ' ,len(probs))
  predicted_classes=[0 if i < 0.5 else 1 for i in probs]
 

  print(classification_report(y_test, predicted_classes,digits=3))
  return (predicted_classes,probs)


In [22]:
prediction_cnnlstm,probs_cnnlstm=bilstm(X_train_initial,X_test_initial,Y_TRAIN_ENCODED_FULL,Y_TEST_ENCODED_FULL)

missed_words : 2607
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30)]              0         
                                                                 
 embedding_6 (Embedding)     (None, 30, 300)           5155800   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              439296    
 l)                                                              
                                                                 
 dense_12 (Dense)            (None, 128)               32896     
                                                                 
 dense_13 (Dense)            (None, 1)                 129       
                                                                 
Total params: 5,628,121
Trainable params: 472,321
Non-trainable params: 5,155,800
_________________________

In [23]:
from sklearn.metrics import f1_score
import pickle

def lstm(vocab_size,X_train,X_test,y_train,y_test,wordIndex,trainable):
  early_stopping = [EarlyStopping(monitor='val_loss',
                        min_delta=0,restore_best_weights=True,
                        patience=5,
                        verbose=1, mode='auto')]
  model = Sequential()
  model.add(createEmbeddingLayer(wordIndex,trainable))
  model.add(Dropout(0.2))
  # model.add(Conv1D(128, 1, activation='relu'))
  # model.add(Conv1D(128, 3, activation='relu'))

  model.add(MaxPooling1D(pool_size=2))
  model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
  # model.add(layers.Dense(100, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.summary()
  model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False), metrics=['acc',f1_m,precision_m, recall_m])
  ## Fit the model
  model.fit(X_train, y_train, validation_split=0.1, epochs=20,callbacks=early_stopping,batch_size=32)
  loss, accuracy, f1_score, precision, recall = model.evaluate(X_train, y_train, verbose=1)
  print("LSTM Training Loss: {:.4f}".format(loss))
  print("LSTM Training Accuracy: {:.4f}".format(accuracy))
  print("LSTM Training f1 score: {:.4f}".format(f1_score))
  print("LSTM Training Precision: {:.4f}".format(precision))
  print("LSTM Training Recall: {:.4f}".format(recall))

  loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)
  print("LSTM Test Loss: {:.4f}".format(loss))
  print("LSTM Test Accuracy: {:.4f}".format(accuracy))
  print("LSTM Test f1 score: {:.4f}".format(f1_score))
  print("LSTM Test Precision: {:.4f}".format(precision))
  print("LSTM Test Recall: {:.4f}".format(recall))

  probs = model.predict(X_test_initial, verbose=1)
  print('lenght of probs : ' ,len(probs))
  predicted_classes=[0 if i < 0.5 else 1 for i in probs]


  # filename = 'finalized_model_lstm.sav'
  # pickle.dump(model, open(filename, 'wb'))

  print(classification_report(y_test, predicted_classes ,digits=3 ))


  print("LSTM  ends..")
  return (predicted_classes,probs)

prediction_cnnlstm,probs_cnnlstm=lstm(vocab_size_initial,X_train_initial,X_test_initial,Y_TRAIN_ENCODED_FULL,Y_TEST_ENCODED_FULL,wordIndex_initial,True)



missed_words : 2607
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 300)         5155800   
                                                                 
 dropout_7 (Dropout)         (None, None, 300)         0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 300)        0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense_14 (Dense)            (None, 1)                 101       
                                                                 
Total params: 5,316,301
Trainable params: 5,316,301
Non-trainable params: 0
________________________